In [ ]:
import pandas as pd
import numpy as np
import matplotlib.ticker as mtick

df = pd.read_csv('Travel.csv')
df.head()

,CustomerID,ProdTaken,Age,TypeofContact,CityTier,DurationOfPitch,Occupation,Gender,NumberOfPersonVisiting,NumberOfFollowups,ProductPitched,PreferredPropertyStar,MaritalStatus,NumberOfTrips,Passport,PitchSatisfactionScore,OwnCar,NumberOfChildrenVisiting,Designation,MonthlyIncome
0,200000,1,41.0,Self Enquiry,3,6.0,Salaried,Female,3,3.0,Deluxe,3.0,Single,1.0,1,2,1,0.0,Manager,20993.0
1,200001,0,49.0,Company Invited,1,14.0,Salaried,Male,3,4.0,Deluxe,4.0,Divorced,2.0,0,3,1,2.0,Manager,20130.0
2,200002,1,37.0,Self Enquiry,1,8.0,Free Lancer,Male,3,4.0,Basic,3.0,Single,7.0,1,3,0,0.0,Executive,17090.0
3,200003,0,33.0,Company Invited,1,9.0,Salaried,Female,2,3.0,Basic,3.0,Divorced,2.0,1,5,1,1.0,Executive,17909.0
4,200004,0,NaN,Self Enquiry,1,8.0,Small Business,Male,2,3.0,Basic,4.0,Divorced,1.0,0,5,1,0.0,Executive,18468.0


In [ ]:
df.shape

(4888, 20)

In [ ]:
df_selected = df[['Designation', 'Passport', 'CityTier', 'MaritalStatus', 
                  'Occupation', 'MonthlyIncome',  'Age', 'PreferredPropertyStar'
                  ,'NumberOfFollowups', 'NumberOfPersonVisiting', 'NumberOfChildrenVisiting', 
                  'ProductPitched']].copy()
df_selected.head(3)

,Designation,Passport,CityTier,MaritalStatus,Occupation,MonthlyIncome,Age,PreferredPropertyStar,NumberOfFollowups,NumberOfPersonVisiting,NumberOfChildrenVisiting,ProductPitched
0,Manager,1,3,Single,Salaried,20993.0,41.0,3.0,3.0,3,0.0,Deluxe
1,Manager,0,1,Divorced,Salaried,20130.0,49.0,4.0,4.0,3,2.0,Deluxe
2,Executive,1,1,Single,Free Lancer,17090.0,37.0,3.0,4.0,3,0.0,Basic


In [ ]:
df_selected.isnull().sum(axis=0)

Designation                   0
Passport                      0
CityTier                      0
MaritalStatus                 0
Occupation                    0
MonthlyIncome               233
Age                         226
PreferredPropertyStar        26
NumberOfFollowups            45
NumberOfPersonVisiting        0
NumberOfChildrenVisiting     66
ProductPitched                0
dtype: int64

In [ ]:
# Replaced with mean
df_selected['Age'].fillna(round(df['Age'].mean()), inplace=True)
df_selected['MonthlyIncome'].fillna(round(df['MonthlyIncome'].mean()), inplace=True)

# Replaced with median
df_selected['NumberOfFollowups'].fillna(df['NumberOfFollowups'].median(), inplace=True)

# Replaced with mode
df_selected['NumberOfChildrenVisiting'].fillna(df['NumberOfChildrenVisiting'].mode()[0], inplace=True)
df_selected['PreferredPropertyStar'].fillna(df['PreferredPropertyStar'].mode()[0], inplace=True)

In [ ]:
df_selected.isnull().sum(axis=0)

Designation                 0
Passport                    0
CityTier                    0
MaritalStatus               0
Occupation                  0
MonthlyIncome               0
Age                         0
PreferredPropertyStar       0
NumberOfFollowups           0
NumberOfPersonVisiting      0
NumberOfChildrenVisiting    0
ProductPitched              0
dtype: int64

In [ ]:
df_selected.duplicated().sum()

395

In [ ]:
print('Duplicate Checking')
print('Before duplicate rows = ' + str(df_selected.shape))
df_selected = df_selected.drop_duplicates()
print('After duplicate rows = ' + str(df_selected.shape))

Duplicate Checking
Before duplicate rows = (4888, 12)
After duplicate rows = (4493, 12)


In [ ]:
nums = ['MonthlyIncome', 'Age', 'PreferredPropertyStar', 'NumberOfFollowups', 'NumberOfPersonVisiting', 'NumberOfChildrenVisiting']

In [ ]:
from scipy import stats

print(f'Jumlah baris sebelum memfilter outlier: {len(df_selected)}')

filtered_entries = np.array([True] * len(df_selected))

for col in ['MonthlyIncome', 'NumberOfFollowups', 'NumberOfPersonVisiting']:
    zscore = abs(stats.zscore(df_selected[col])) # hitung absolute z-scorenya
    filtered_entries = (zscore < 3) & filtered_entries # keep yang kurang dari 3 absolute z-scorenya
    
df_selected = df_selected[filtered_entries] # filter, cuma ambil yang z-scorenya dibawah 3

print(f'Jumlah baris setelah memfilter outlier: {len(df_selected)}')

Jumlah baris sebelum memfilter outlier: 4493
Jumlah baris setelah memfilter outlier: 4489


In [ ]:
def segment(x):
    if x['Passport'] == 1 and x['CityTier'] == 3 and ((x['MaritalStatus'] == 'Single') or (x['MaritalStatus'] == 'Unmarried')) and x['Occupation'] == 'Large Business':
        chance = 'High'
    elif (x['MonthlyIncome'] >= 15000 and x['MonthlyIncome'] <= 25000) and (x['Age'] >= 15 and x['Age'] <= 30) and x['PreferredPropertyStar'] == 5:
        chance = 'High'
    else:
        chance = 'Low'
    return chance

In [ ]:
df_clean = df_selected.copy()

df_clean['Target'] = df_clean.apply(lambda x: segment(x), axis=1) 
df_clean

,Designation,Passport,CityTier,MaritalStatus,Occupation,MonthlyIncome,Age,PreferredPropertyStar,NumberOfFollowups,NumberOfPersonVisiting,NumberOfChildrenVisiting,ProductPitched,Target
0,Manager,1,3,Single,Salaried,20993.0,41.0,3.0,3.0,3,0.0,Deluxe,Low
1,Manager,0,1,Divorced,Salaried,20130.0,49.0,4.0,4.0,3,2.0,Deluxe,Low
2,Executive,1,1,Single,Free Lancer,17090.0,37.0,3.0,4.0,3,0.0,Basic,Low
3,Executive,1,1,Divorced,Salaried,17909.0,33.0,3.0,3.0,2,1.0,Basic,Low
4,Executive,0,1,Divorced,Small Business,18468.0,38.0,4.0,3.0,2,0.0,Basic,Low
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4883,Manager,1,3,Unmarried,Small Business,26576.0,49.0,4.0,5.0,3,1.0,Deluxe,Low
4884,Executive,1,1,Single,Salaried,21212.0,28.0,3.0,5.0,4,2.0,Basic,Low
4885,Senior Manager,0,3,Married,Salaried,31820.0,52.0,4.0,4.0,4,3.0,Standard,Low
4886,Executive,0,3,Single,Small Business,20289.0,19.0,3.0,4.0,3,2.0,Basic,Low


In [ ]:
df_clean['Target'] = df_clean['Target'].apply(lambda x : 1 if x == 'High' else 0)

In [ ]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4489 entries, 0 to 4887
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Designation               4489 non-null   object 
 1   Passport                  4489 non-null   int64  
 2   CityTier                  4489 non-null   int64  
 3   MaritalStatus             4489 non-null   object 
 4   Occupation                4489 non-null   object 
 5   MonthlyIncome             4489 non-null   float64
 6   Age                       4489 non-null   float64
 7   PreferredPropertyStar     4489 non-null   float64
 8   NumberOfFollowups         4489 non-null   float64
 9   NumberOfPersonVisiting    4489 non-null   int64  
 10  NumberOfChildrenVisiting  4489 non-null   float64
 11  ProductPitched            4489 non-null   object 
 12  Target                    4489 non-null   int64  
dtypes: float64(5), int64(4), object(4)
memory usage: 491.0+ KB


In [ ]:
features = ['Passport','CityTier','MonthlyIncome','PreferredPropertyStar','NumberOfFollowups','NumberOfPersonVisiting','NumberOfChildrenVisiting']

In [ ]:
X = df_clean[features] # menggunakan semua feature kecuali target
y = df_clean['Target']

In [ ]:
#Splitting the data into Train and Test
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

# KNN

In [ ]:
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)
y_proba = y_proba[:,1]
eval_classification(model, y_pred, y_proba, X_train, y_train, X_test, y_test)

NameError: name 'model' is not defined

# Random Forrest

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor()
rf.fit(X_train, y_train)
pred = rf.predict(X_test)
eval_regression(rf, pred, X_train, y_train, X_test, y_test)

NameError: name 'eval_regression' is not defined

# Lasso 

In [ ]:
from sklearn.linear_model import Lasso
lasso_model = Lasso()
lasso_model.fit(X_train, y_train)

pred = lasso_model.predict(X_test)
eval_regression(lasso_model, pred, X_train, y_train, X_test, y_test)

# Ridge

In [ ]:
from sklearn.linear_model import Lasso
lasso_model = Lasso()
lasso_model.fit(X_train, y_train)

pred = lasso_model.predict(X_test)
eval_regression(lasso_model, pred, X_train, y_train, X_test, y_test)

# Elastic Net

In [ ]:
from sklearn.linear_model import ElasticNet

elasticnet_model = ElasticNet()
elasticnet_model.fit(X_train, y_train)

pred = elasticnet_model.predict(X_test)
eval_regression(elasticnet_model, pred, X_train, y_train, X_test, y_test)

# Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(random_state=42)
model.fit(X_train,y_train)

y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)
y_proba = y_proba[:,1]
eval_classification(model, y_pred, y_proba, X_train, y_train, X_test, y_test)

In [ ]:
print('Train score: ' + str(model.score(X_train, y_train))) #accuracy
print('Test score:' + str(model.score(X_test, y_test))) #accuracy

# Decision Tree

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
def eval_regression(model, pred, xtrain, ytrain, xtest, ytest):
    print("MAE: %.2f" % mean_absolute_error(ytest, pred)) # The MAE
    print("RMSE: %.2f" % mean_squared_error(ytest, pred, squared=False)) # The RMSE
    print('R2 score: %.2f' % r2_score(ytest, pred)) # Explained variance score: 1 is perfect prediction
from sklearn.tree import DecisionTreeRegressor

dt = DecisionTreeRegressor()
dt.fit(X_train, y_train)
pred = dt.predict(X_test)
eval_regression(dt, pred, X_train, y_train, X_test, y_test)

MAE: 0.04
RMSE: 0.20
R2 score: -0.06


In [ ]:
# Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression() 

logreg.fit(X_train, y_train)

LogisticRegression()

In [ ]:
pred = logreg.predict(X_test)

In [ ]:
print('Coefficients: \n', logreg.coef_) # The slope
print('Intercept: \n', logreg.intercept_) # The Intercept

Coefficients: 
 [[-1.41570965e-09 -7.94575401e-09 -1.30757085e-04  1.26703477e-09
  -1.48589183e-08 -1.31569981e-08 -4.44632475e-09]]
Intercept: 
 [-4.62862712e-09]


In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
def eval_regression(model, pred, xtrain, ytrain, xtest, ytest):
    print("MAE: %.2f" % mean_absolute_error(ytest, pred)) # The MAE
    print("RMSE: %.2f" % mean_squared_error(ytest, pred, squared=False)) # The RMSE
    print('R2 score: %.2f' % r2_score(ytest, pred)) # Explained variance score: 1 is perfect prediction

eval_regression(logreg, pred, X_train, y_train, X_test,y_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression() 

logreg.fit(X_train, y_train)

pred = logreg.predict(X_test)
print('Coefficients: \n', logreg.coef_) # The slope
print('Intercept: \n', logreg.intercept_) # The Intercept

eval_regression(logreg, pred, X_train, y_train, X_test,y_test)

MAE: 0.04
RMSE: 0.20
R2 score: -0.04
